In [0]:
from pyspark.sql.functions import from_unixtime, col, round, to_date

In [0]:
df = spark.table("steam.bronze.user_games")

In [0]:
# Converte para timestamp
df = df.withColumn(
    'rtime_last_played',
    from_unixtime(col('rtime_last_played')).cast('timestamp')
)

# Conversão minutos → horas com 2 casas decimais
df = df.withColumn(
    'playtime_forever',
    round(col('playtime_forever') / 60, 2).cast('double')
)

# Filtrar apenas valores maiores que 0.00
df = df.filter(col('playtime_forever') > 0.00)

df = df.select('appid','playtime_forever','rtime_last_played')

In [0]:
spark.sql(
    """
    CREATE SCHEMA IF NOT EXISTS steam.silver
    """
)

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("steam.silver.user_games")